In [ ]:
# imports
import pymysql
pymysql.install_as_MySQLdb()
from sqlalchemy import create_engine
from sqlalchemy.types import *
from sqlalchemy_utils import database_exists
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import statsmodels.stats.multicomp as mc

# Questions to Answer


## Q1: Does the MPAA rating of a movie (G/PG/PG-13/R) affect how much revenue the movie generates?



### State your Null Hypothesis and Alternative Hypothesis

Null Hypothesis: No difference between mpaa rating and movie revenue.

Alternative Hypothesis: There is difference between mpaa rating and movie revenue.

### Select the correct test according to the data type and number of samples

it is a four sample numercial data, so I will perform ANOVA and/or Tukey test.



### Test the assumptions of your selected test

### Test Execution (Kruskal-Wallis)

### p-value Interpretation

### Test Execution (Kruskal-Wallis)

### p-value Interpretation

### supporting visualization 